In [1]:
import scanpy as sc
data_dir = '../../dataset/annotated_Chromium_FFPE_Human_Breast_Cancer_Chromium_FFPE_Human_Breast_Cancer_count_sample_filtered_feature_bc_matrix.h5ad'
adata = sc.read_h5ad(data_dir)
print(adata.X)
sc.pp.normalize_total(adata,target_sum=1e4)
sc.pp.log1p(adata)

/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 71173076 stored elements and shape (27460, 18082)>
  Coords	Values
  (0, 13)	1.0
  (0, 40)	1.0
  (0, 48)	1.0
  (0, 108)	1.0
  (0, 114)	1.0
  (0, 146)	1.0
  (0, 171)	1.0
  (0, 225)	1.0
  (0, 235)	3.0
  (0, 261)	1.0
  (0, 262)	3.0
  (0, 263)	2.0
  (0, 276)	1.0
  (0, 280)	5.0
  (0, 366)	1.0
  (0, 369)	1.0
  (0, 378)	2.0
  (0, 391)	1.0
  (0, 402)	1.0
  (0, 438)	1.0
  (0, 439)	1.0
  (0, 452)	1.0
  (0, 471)	1.0
  (0, 478)	1.0
  (0, 516)	1.0
  :	:
  (27459, 17744)	2.0
  (27459, 17747)	2.0
  (27459, 17764)	1.0
  (27459, 17769)	1.0
  (27459, 17780)	1.0
  (27459, 17812)	1.0
  (27459, 17820)	1.0
  (27459, 17843)	1.0
  (27459, 17867)	2.0
  (27459, 17899)	1.0
  (27459, 17901)	1.0
  (27459, 17911)	1.0
  (27459, 17915)	2.0
  (27459, 17919)	1.0
  (27459, 17960)	1.0
  (27459, 17961)	1.0
  (27459, 17964)	1.0
  (27459, 17969)	1.0
  (27459, 17971)	1.0
  (27459, 18008)	1.0
  (27459, 18027)	2.0
  (27459, 18030)	1.0
  (27459, 18035)	2.0
  (27459, 

In [2]:
import torch
import lightning.pytorch as pl
from self_supervision.models.lightning_modules.cellnet_autoencoder import MLPAutoEncoder
from self_supervision.estimator.cellnet import EstimatorAutoEncoder

# 设置你的 .ckpt 文件路径
ckpt_path = "../../sc_pretrained/Pretrained Models/GPMask.ckpt"

# 模型参数
units_encoder = [512, 512, 256, 256, 64]
units_decoder = [256, 256, 512, 512]

# 初始化 EstimatorAutoEncoder 实例
estim = EstimatorAutoEncoder(data_path=None)  # 如果没有实际数据路径，可以设置为None

# 加载预训练模型
estim.model = MLPAutoEncoder.load_from_checkpoint(
    ckpt_path,
    gene_dim=19331,  # 根据你的数据调整
    batch_size=2048,  # 根据你的需要调整
    units_encoder=units_encoder, 
    units_decoder=units_decoder,
    masking_strategy="random",  # 假设模型使用了随机掩码
    masking_rate=0.5,  # 根据需要调整
)

# 使用 GPU 进行评估（如果可用）
estim.trainer = pl.Trainer(accelerator="gpu", devices=1 if torch.cuda.is_available() else None)
estim.model

/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs


MLPAutoEncoder(
  (train_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=test_
  )
  (encoder): MLP(
    (0): Linear(in_features=19331, out_features=512, bias=True)
    (1): SELU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SELU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SELU()
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=256, out_features=256, b

In [3]:
import pandas as pd
var_df = pd.read_parquet('../../sc_pretrained/var.parquet')
var_df

,feature_id,feature_name
0,ENSG00000186092,OR4F5
1,ENSG00000284733,OR4F29
2,ENSG00000284662,OR4F16
3,ENSG00000187634,SAMD11
4,ENSG00000188976,NOC2L
...,...,...
19326,ENSG00000288702,UGT1A3
19327,ENSG00000288705,UGT1A5
19328,ENSG00000182484,WASH6P
19329,ENSG00000288622,PDCD6-AHRR


In [4]:
all_genes = var_df['feature_name'].tolist()
all_genes

['OR4F5',
 'OR4F29',
 'OR4F16',
 'SAMD11',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'PERM1',
 'HES4',
 'ISG15',
 'AGRN',
 'RNF223',
 'C1orf159',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'C1QTNF12',
 'UBE2J2',
 'SCNN1D',
 'ACAP3',
 'PUSL1',
 'INTS11',
 'CPTP',
 'TAS1R3',
 'DVL1',
 'MXRA8',
 'AURKAIP1',
 'CCNL2',
 'MRPL20',
 'ANKRD65',
 'TMEM88B',
 'VWA1',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'TMEM240',
 'SSU72',
 'FNDC10',
 'MIB2',
 'MMP23B',
 'CDK11B',
 'SLC35E2B',
 'CDK11A',
 'NADK',
 'GNB1',
 'CALML6',
 'TMEM52',
 'CFAP74',
 'GABRD',
 'PRKCZ',
 'FAAP20',
 'SKI',
 'MORN1',
 'RER1',
 'PEX10',
 'PLCH2',
 'PANK4',
 'HES5',
 'TNFRSF14',
 'PRXL2B',
 'MMEL1',
 'TTC34',
 'ACTRT2',
 'PRDM16',
 'ARHGEF16',
 'MEGF6',
 'TPRG1L',
 'WRAP73',
 'TP73',
 'CCDC27',
 'SMIM1',
 'LRRC47',
 'CEP104',
 'DFFB',
 'C1orf174',
 'AJAP1',
 'NPHP4',
 'KCNAB2',
 'CHD5',
 'RPL22',
 'RNF207',
 'ICMT',
 'HES3',
 'GPR153',
 'ACOT7',
 'HES2',
 'ESPN',
 'TNFRSF25',
 'PLEKHG5',
 'NOL9',
 'TAS1R1',
 'ZBTB48',
 'KLH

In [5]:
adata.var['gene_name']=adata.var.index
adata.var['gene_name']

SAMD11      SAMD11
NOC2L        NOC2L
KLHL17      KLHL17
PLEKHN1    PLEKHN1
PERM1        PERM1
            ...   
MT-ND4L    MT-ND4L
MT-ND4      MT-ND4
MT-ND5      MT-ND5
MT-ND6      MT-ND6
MT-CYB      MT-CYB
Name: gene_name, Length: 18082, dtype: object

In [6]:
import numpy as np
# 初始化一个新的数据矩阵，形状为 (adata.X.shape[0], len(all_genes))，填充为零
new_data = np.zeros((adata.X.shape[0], len(all_genes)), dtype=np.float32)

In [7]:
existing_genes = adata.var['gene_name']
existing_genes

SAMD11      SAMD11
NOC2L        NOC2L
KLHL17      KLHL17
PLEKHN1    PLEKHN1
PERM1        PERM1
            ...   
MT-ND4L    MT-ND4L
MT-ND4      MT-ND4
MT-ND5      MT-ND5
MT-ND6      MT-ND6
MT-CYB      MT-CYB
Name: gene_name, Length: 18082, dtype: object

In [8]:
# 将所有基因名称转换为小写
all_genes_lower = [gene.lower() for gene in all_genes]
adata_genes_lower = [gene.lower() for gene in existing_genes]

# 将两个列表转换为集合
all_genes_set = set(all_genes_lower)
adata_genes_set = set(adata_genes_lower)

# 计算交集
matching_genes = all_genes_set.intersection(adata_genes_set)
matching_count = len(matching_genes)
# 计算不匹配的基因
non_matching_genes = adata_genes_set - matching_genes
non_matching_count = len(non_matching_genes)


# 输出结果
print(f"匹配的基因数量: {matching_count}")
print(f"匹配的基因列表: {matching_genes}")
non_matching_genes


匹配的基因数量: 17768
匹配的基因列表: {'farp1', 'rbm14', 'glrx', 'itga7', 'smg1', 'itga10', 'sox13', 'spon2', 'wfdc6', 'rnf31', 'clic2', 'ttc17', 'mpp2', 'adamts19', 'klf6', 'sfxn1', 'dnajc30', 'c22orf31', 'slc39a2', 'znf416', 'kiaa0825', 'ccl11', 'sike1', 'mthfs', 'prpf40b', 'lrrc4c', 'hacl1', 'il18', 'snrnp70', 'eipr1', 'rtp4', 'ddo', 'bri3', 'cdh10', 'fam237b', 'tns3', 'cracr2a', 'dnajc5', 'svbp', 'fxyd6', 'tmprss7', 'tspyl1', 'ppm1m', 'ppme1', 'zscan26', 'cul2', 'angel1', 'lgi4', 'shc2', 'clec2a', 'mllt11', 'mreg', 'pcdh8', 'kcnj14', 'smtn', 'znf362', 'znf57', 'rhobtb3', 'chst15', 'tmem186', 'bag5', 'arhgef25', 'naa50', 'fscn1', 'ddx59', 'mtor', 'abhd17b', 'senp5', 'synpo2', 'nrbf2', 'abi3bp', 'tas2r40', 'prr23b', 'gp9', 'prkacg', 'isg15', 'mfap3l', 'chrdl1', 'usp31', 'or5d3p', 'ift22', 'nbpf19', 'nip7', 'dnttip1', 'siae', 'hmx2', 'cnnm1', 'sec13', 'gpc1', 'lrrc59', 'gpm6b', 'ferd3l', 'acox1', 'ripor3', 'dusp26', 'kyat1', 'il5', 'git1', 'ubxn2a', 'acsf3', 'il1rapl1', 'psip1', 'znhit1', 'scrt1', 

{'aars',
 'ac004593.3',
 'ac005551.1',
 'ac007244.1',
 'ac007906.2',
 'ac008397.1',
 'ac008687.4',
 'ac010255.3',
 'ac010325.1',
 'ac011005.1',
 'ac011195.2',
 'ac013470.2',
 'ac015871.1',
 'ac021072.1',
 'ac021097.2',
 'ac025283.2',
 'ac025287.4',
 'ac067752.1',
 'ac072022.2',
 'ac073111.4',
 'ac087498.1',
 'ac090360.1',
 'ac091057.6',
 'ac092835.1',
 'ac093323.1',
 'ac099489.1',
 'ac104389.5',
 'ac106774.4',
 'ac113348.1',
 'ac115220.1',
 'ac118549.1',
 'ac119396.1',
 'ac132217.2',
 'ac134684.8',
 'ac135068.1',
 'ac138647.1',
 'ac187653.1',
 'ac233723.1',
 'ac236972.4',
 'acpp',
 'adprhl2',
 'adss',
 'adssl1',
 'al032819.3',
 'al109810.2',
 'al121578.2',
 'al135905.2',
 'al160269.1',
 'al162231.1',
 'al162596.1',
 'al353572.3',
 'al354761.1',
 'al391650.1',
 'al445238.1',
 'al451007.3',
 'al590560.2',
 'al603764.2',
 'al772284.2',
 'al845331.2',
 'alg1l',
 'ap000552.4',
 'ap002495.1',
 'arih2os',
 'armc4',
 'arse',
 'atp5md',
 'atp5mpl',
 'atp6ap1l',
 'b3gnt10',
 'bhmg1',
 'bx255925.

In [9]:
gene_to_index = {gene: idx for idx, gene in enumerate(all_genes_lower)}
gene_to_index

{'or4f5': 0,
 'or4f29': 1,
 'or4f16': 2,
 'samd11': 3,
 'noc2l': 4,
 'klhl17': 5,
 'plekhn1': 6,
 'perm1': 7,
 'hes4': 8,
 'isg15': 9,
 'agrn': 10,
 'rnf223': 11,
 'c1orf159': 12,
 'ttll10': 13,
 'tnfrsf18': 14,
 'tnfrsf4': 15,
 'sdf4': 16,
 'b3galt6': 17,
 'c1qtnf12': 18,
 'ube2j2': 19,
 'scnn1d': 20,
 'acap3': 21,
 'pusl1': 22,
 'ints11': 23,
 'cptp': 24,
 'tas1r3': 25,
 'dvl1': 26,
 'mxra8': 27,
 'aurkaip1': 28,
 'ccnl2': 29,
 'mrpl20': 30,
 'ankrd65': 31,
 'tmem88b': 32,
 'vwa1': 33,
 'atad3c': 34,
 'atad3b': 35,
 'atad3a': 36,
 'tmem240': 37,
 'ssu72': 38,
 'fndc10': 39,
 'mib2': 40,
 'mmp23b': 41,
 'cdk11b': 42,
 'slc35e2b': 43,
 'cdk11a': 44,
 'nadk': 45,
 'gnb1': 46,
 'calml6': 47,
 'tmem52': 48,
 'cfap74': 49,
 'gabrd': 50,
 'prkcz': 51,
 'faap20': 52,
 'ski': 53,
 'morn1': 54,
 'rer1': 55,
 'pex10': 56,
 'plch2': 57,
 'pank4': 58,
 'hes5': 59,
 'tnfrsf14': 60,
 'prxl2b': 61,
 'mmel1': 62,
 'ttc34': 63,
 'actrt2': 64,
 'prdm16': 65,
 'arhgef16': 66,
 'megf6': 67,
 'tprg1l': 68

In [10]:
only_in_all_genes = all_genes_set - adata_genes_set

only_in_adata_genes = adata_genes_set - all_genes_set

# 输出结果
print(f"仅在 all_genes 中存在的基因数量: {len(only_in_all_genes)}")
print(f"仅在 all_genes 中存在的基因: {only_in_all_genes}")

print(f"仅在 adata_genes 中存在的基因数量: {len(only_in_adata_genes)}")
print(f"仅在 adata_genes 中存在的基因: {only_in_adata_genes}")


仅在 all_genes 中存在的基因数量: 1563
仅在 all_genes 中存在的基因: {'plglb1', 'igll5', 'h3c11', 'gcom1', 'rps9', 'gage12f', 'foxd4l3', 'fam86b2', 'mtrnr2l10', 'mrps9', 'poteh', 'atp5mk', 'znf737', 'stip1', 'pcdhgb1', 'mrpl16', 'smim11b', 'rpl4', 'isy1-rab43', 'satl1', 'rbm4', 'golga4', 'rps4y1', 'coro7-pam16', 'rpl13', 'nutm2g', 'dus4l-bcap29', 'tlcd4-rwdd3', 'rps6kc1', 'ak2', 'bnip3', 'kcnk9', 'ptcd3', 'h2bc7', 'cntf', 'rpl23a', 'hla-dpb1', 'pgap4', 'h2ac18', 'park7', 'fam184b', 'npipa8', 'fmc1-luc7l2', 'ppfia1', 's100a7', 'gdf1', 'hla-c', 'h2bc10', 'nphp3-acad11', 'arsl', 'mrps30', 'ugt2b15', 'scrg1', 'fcn1', 'bola2', 'rps3', 'golga6l10', 'cers1', 'bcas1', 'srrm3', 'gtf2h2', 'gpx1', 'scygr6', 'meiosin', 'fam90a22p', 'ankrd20a1', 'or51i1', 'fam90a9p', 'mag', 'mrpl39', 'cilk1', 'cdy2a', 'h1-2', 'grapl', 'znf816-znf321p', 'ankrd20a4p', 'h3y1', 'eif3c', 'h2bc5', 'ovca2', 'znf487', 'or4f16', 'ly6g6f-ly6g6d', 'saa2-saa4', 'rpl41', 'iho1', 'mrpl19', 'tmem191c', 'ckmt1a', 'ctage9', 'ankrd20a2p', 'csag3', 'tbc

In [11]:
dense_adata_X = adata.X.toarray()
for i, gene in enumerate(adata_genes_lower):
    if gene in gene_to_index:
        new_data[:, gene_to_index[gene]] = dense_adata_X[:, i]
    else:
        print(f'Gene {gene} not found in all_genes list')

Gene al391650.1 not found in all_genes list
Gene yars not found in all_genes list
Gene adprhl2 not found in all_genes list
Gene tctex1d4 not found in all_genes list


Gene tctex1d1 not found in all_genes list
Gene wdr78 not found in all_genes list
Gene hhla3 not found in all_genes list
Gene ac118549.1 not found in all_genes list
Gene wdr63 not found in all_genes list
Gene kiaa1324 not found in all_genes list
Gene sars not found in all_genes list
Gene hist2h2be not found in all_genes list


Gene al162596.1 not found in all_genes list
Gene lor not found in all_genes list
Gene c1orf61 not found in all_genes list
Gene al590560.2 not found in all_genes list
Gene rgs5 not found in all_genes list
Gene dusp27 not found in all_genes list


Gene eprs not found in all_genes list
Gene marc2 not found in all_genes list
Gene marc1 not found in all_genes list
Gene h3f3a not found in all_genes list
Gene hist3h3 not found in all_genes list
Gene hist3h2a not found in all_genes list
Gene hist3h2bb not found in all_genes list
Gene al109810.2 not found in all_genes list
Gene tbce not found in all_genes list
Gene tbce not found in all_genes list
Gene adss not found in all_genes list
Gene al451007.3 not found in all_genes list
Gene gcsaml-as1 not found in all_genes list
Gene mycnos not found in all_genes list
Gene fam49a not found in all_genes list


Gene c2orf91 not found in all_genes list
Gene igkc not found in all_genes list
Gene al845331.2 not found in all_genes list
Gene ac092835.1 not found in all_genes list
Gene kiaa1211l not found in all_genes list


...

Gene siglec5 not found in all_genes list


Gene gdf5os not found in all_genes list
Gene tmem189 not found in all_genes list
Gene fp565260.1 not found in all_genes list


Gene ap000552.4 not found in all_genes list
Gene iglc1 not found in all_genes list
Gene iglc7 not found in all_genes list
Gene lrp5l not found in all_genes list
Gene elfn2 not found in all_genes list
Gene h1f0 not found in all_genes list
Gene z82206.1 not found in all_genes list


Gene arse not found in all_genes list
Gene cxorf21 not found in all_genes list
Gene hypm not found in all_genes list
Gene al121578.2 not found in all_genes list
Gene bx276092.9 not found in all_genes list


Gene nxf5 not found in all_genes list
Gene glra4 not found in all_genes list
Gene tmsb15b not found in all_genes list
Gene h2bfwt not found in all_genes list
Gene h2bfm not found in all_genes list
Gene tmsb15b not found in all_genes list
Gene pih1d3 not found in all_genes list
Gene al772284.2 not found in all_genes list
Gene cxorf56 not found in all_genes list
Gene fam122b not found in all_genes list
Gene fam122c not found in all_genes list
Gene cxorf40a not found in all_genes list
Gene ac236972.4 not found in all_genes list
Gene prky not found in all_genes list
Gene ac007244.1 not found in all_genes list


In [12]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split


label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(adata.obs['cell_type'])  # 预先编码标签


random_seed = 42
X_train_val, X_test, y_train_val, y_test = train_test_split(
    new_data, labels_encoded, test_size=0.15, random_state=random_seed)


X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765, random_state=random_seed)  # 0.1765 是为了让验证集占 15%

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

estim.model.eval()
with torch.no_grad():
    X_train_tensor = torch.tensor(X_train).float().to(device)
    X_test_tensor = torch.tensor(X_test).float().to(device)
    train_embeddings = estim.model.encoder(X_train_tensor).detach().cpu().numpy()
    test_embeddings = estim.model.encoder(X_test_tensor).detach().cpu().numpy()


cuda


In [13]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

    

    # 初始化和训练KNN分类器
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_embeddings, y_train)
    
    # 模型预测
predictions = knn.predict(test_embeddings)

    # 计算准确率和 F1 分数
accuracy = accuracy_score(y_test, predictions)
print(f"KNN Accuracy on Test Data: {accuracy}")
f1 = f1_score(y_test, predictions, average='weighted')
print(f"Weighted F1 Score: {f1}")
    
macro_f1 = f1_score(y_test, predictions, average='macro')
print(f'Macro F1 Score: {macro_f1}')

    # 计算随机猜测的准确率
class_probabilities = np.bincount(y_test) / len(y_test)
random_accuracy = np.sum(class_probabilities ** 2)
print(f"Random Guess Accuracy: {random_accuracy}")

    # 生成分类报告
report = classification_report(y_test, predictions, target_names=label_encoder.classes_)
print(report)

KNN Accuracy on Test Data: 0.7280893420733188
Weighted F1 Score: 0.7252529229909821
Macro F1 Score: 0.7262097790842063
Random Guess Accuracy: 0.08554398369790313
                         precision    recall  f1-score   support

                B Cells       0.81      0.74      0.77       219
           CD4+ T Cells       0.79      0.92      0.85       392
           CD8+ T Cells       0.86      0.83      0.84       281
                 DCIS 1       0.45      0.51      0.48       293
                 DCIS 2       0.53      0.48      0.50       326
            Endothelial       0.92      0.82      0.87       154
              IRF7+ DCs       0.96      0.76      0.85        33
         Invasive Tumor       0.65      0.85      0.74       701
             LAMP3+ DCs       0.90      0.95      0.92        19
          Macrophages 1       0.86      0.71      0.78       425
          Macrophages 2       0.82      0.84      0.83       106
             Mast Cells       0.87      1.00      0.93   

In [14]:
with torch.no_grad():
    new_data_tensor = torch.tensor(new_data).float().to(device)
    SSL_embeddings = estim.model.encoder(new_data_tensor).detach().cpu().numpy()
new_adata = sc.read_h5ad(data_dir)
new_adata.obsm[f'SSL_GP_ZS_{random_seed}'] = SSL_embeddings
new_adata.uns[f'GP_ZS_y_test_{random_seed}'] = y_test
new_adata.uns[f'GP_ZS_predictions_{random_seed}'] = predictions
new_adata.uns[f'GP_ZS_target_names_{random_seed}'] = label_encoder.classes_
new_adata.write_h5ad(data_dir)

/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [15]:

import pandas as pd
import os
import re

# 当前 Notebook 文件名
notebook_name = "chromium_breast_cancer_GP_mask_zero_shot_42.ipynb"

# 初始化需要打印的值
init_train_loss = train_losses[0] if 'train_losses' in globals() else None
init_val_loss = val_losses[0] if 'val_losses' in globals() else None
converged_epoch = len(train_losses) - patience if 'train_losses' in globals() else None
converged_val_loss = best_val_loss if 'best_val_loss' in globals() else None

# 打印所有所需的指标
print("Metrics Summary:")
if 'train_losses' in globals():
    print(f"init_train_loss\tinit_val_loss\tconverged_epoch\tconverged_val_loss\tmacro_f1\tweighted_f1\tmicro_f1")
    print(f"{init_train_loss:.3f}\t{init_val_loss:.3f}\t{converged_epoch}\t{converged_val_loss:.3f}\t{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")
else:
    print(f"macro_f1\tweighted_f1\tmicor_f1")
    print(f"{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")

# 保存结果到 CSV 文件
output_data = {
    'dataset_split_random_seed': [int(random_seed)],
    'dataset': ['chromium_breast_cancer'],
    'method': [re.search(r'breast_cancer_(.*?)_\d+', notebook_name).group(1)],
    'init_train_loss': [init_train_loss if init_train_loss is not None else ''],
    'init_val_loss': [init_val_loss if init_val_loss is not None else ''],
    'converged_epoch': [converged_epoch if converged_epoch is not None else ''],
    'converged_val_loss': [converged_val_loss if converged_val_loss is not None else ''],
    'macro_f1': [macro_f1],
    'weighted_f1': [f1],
    'micor_f1': [accuracy]
}
output_df = pd.DataFrame(output_data)

# 保存到当前目录下名为 results 的文件夹中
if not os.path.exists('results'):
    os.makedirs('results')

csv_filename = f"results/{os.path.splitext(notebook_name)[0]}_results.csv"
output_df.to_csv(csv_filename, index=False)


Metrics Summary:
macro_f1	weighted_f1	micor_f1
0.726	0.725	0.728
